# Dataset Builder

Takes raw data and makes a real dataset ready for preprocessing.

## Importing

In [ ]:
import pandas as pd

In [ ]:
# Get Dataset into pandas dataframe
df_pesagens = pd.read_csv('../data/raw/pesagens2014.csv', sep=',', index_col='PES_ID')
df_rotas = pd.read_csv('../data/raw/rotas.csv', sep=';', index_col='ROTA_ID')

## Info about the raw data

In [ ]:
df_pesagens.info()

In [ ]:
df_rotas.info()

In [ ]:
df_pesagens.head()

In [ ]:
df_rotas.head()

By looking at both data, as well as looking at the [documentation of the data](http://dados.recife.pe.gov.br/dataset/pesagem-de-coletas-de-residuos), we know that the point of connection for both data is `ROTA_ID` attribute. 

We've gotta be sure that all instances from `pesagens` dataset has a valid `ROTA_ID` that is present on `roteirizacao` dataset, for this, we'll aggregate all valid data and non-valid data will be discarded.

## Merging Rotas Data into Pesagens 

Now we're gonna merge both datasets into just one dataset. This is achieved by using the `merge` method from **pandas**, but, as seen in our tests, if we have duplicate indexes we may see a duplication of our data. 

To fix this, we're gonna drop duplicate indexes present on our data, to make sure the merge is successfull and  no duplicate data is present in the merge.

In [ ]:
df_pesagens = df_pesagens.reset_index().drop_duplicates(subset='PES_ID', keep='first').set_index('PES_ID')
df_rotas = df_rotas.reset_index().drop_duplicates(subset='ROTA_ID', keep='first').set_index('ROTA_ID')

Duplicates removed, we can now merge into one dataset.

In [ ]:
df = df_pesagens.merge(df_rotas, on='ROTA_ID')

In [ ]:
df['ROTA_ID'] = pd.to_numeric(df['ROTA_ID'], errors='coerce')

for i in range(len(df['ROTA_ID'])):
    # print(df.loc[i, 'ROTA_ID'])
    if pd.isnull(df.loc[i, 'ROTA_ID']):
        # print('yes')
        df.loc[i, 'ROTA_ID'] = i

print(df['ROTA_ID'])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# ISTO É APENAS UM TESTE PARA TRANSFORMAR OS DADOS DA COLUNA ROTA_ID EM NÚMEROS, MAS NÃO FUNCIONOU POR ENQUANTO
# POIS OS NÚMEROS DOS NaN NÃO SÃO DIFERENTES

import pandas as pd

# Crie um DataFrame de exemplo
data = {'ROTA_ID': ['GOFI1', '411', '548', 'ACE01']}
da = pd.DataFrame(data)

# Crie uma nova coluna para indicar se o valor original era não numérico
da['ORIGINAL_NON_NUMERIC'] = ~da['ROTA_ID'].str.isnumeric()

# Converta os valores da coluna 'ROTA_ID' para números, ignorando os valores não convertíveis
da['ROTA_ID'] = pd.to_numeric(da['ROTA_ID'], errors='coerce')

# Identifique valores originais não numéricos distintos
non_numeric_values = da.loc[da['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'].unique()

# Substitua os valores não numéricos por valores numéricos distintos
non_numeric_mapping = {value: idx for idx, value in enumerate(non_numeric_values)}
da.loc[da['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'] = da.loc[da['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'].map(non_numeric_mapping)
print(non_numeric_mapping)

# Substitua os valores NaN por algum valor específico (por exemplo, -1)
da['ROTA_ID'] = da['ROTA_ID'].fillna(-1)

# da['ROTA_ID'] = pd.to_numeric(da['ROTA_ID'], errors='coerce') #da['ROTA_ID'].apply(convert_and_categorize)
# print(da['ROTA_ID'])

# Identifique valores originais não numéricos distintos
# non_numeric_values = da.loc[~da['ROTA_ID'].apply(pd.api.types.is_numeric_dtype), 'ROTA_ID']#.unique()
# print(non_numeric_values)

# Substitua os valores não numéricos por valores numéricos distintos
# non_numeric_mapping = {value: idx for idx, value in enumerate(non_numeric_values)}
# for i in enumerate(non_numeric_values):
#     print(i)

# da['ROTA_ID'] = da['ROTA_ID'].apply(lambda x: non_numeric_mapping[x] if pd.isna(x) else x)
# da['ROTA_ID'] = da['ROTA_ID'].fillna(-1)
# da['ROTA_ID'].replace(non_numeric_mapping, inplace=True)

print(da)

In [ ]:
# OUTRO TESTE

# import pandas as pd

# # Crie um DataFrame de exemplo
# data = {'ROTA_ID': ['GOFI1', '411', '548', 'ACE01']}
# df = pd.DataFrame(data)

# # Crie uma nova coluna para indicar se o valor original era não numérico
# df['ORIGINAL_NON_NUMERIC'] = ~df['ROTA_ID'].str.isnumeric()

# # Converta os valores da coluna 'ROTA_ID' para números, ignorando os valores não convertíveis
# df['ROTA_ID'] = pd.to_numeric(df['ROTA_ID'], errors='coerce')
# # print(df['ROTA_ID'])
# # Identifique valores originais não numéricos distintos
# non_numeric_values = df.loc[df['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'].unique()
# # print(non_numeric_values)

# # Substitua os valores não numéricos por valores numéricos distintos
# non_numeric_mapping = {value: idx for idx, value in enumerate(non_numeric_values)}
# print(non_numeric_mapping)

# df.loc[df['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'] = df.loc[df['ORIGINAL_NON_NUMERIC'], 'ROTA_ID'].map(non_numeric_mapping)

# # Substitua os valores NaN por algum valor específico (por exemplo, -1)
# df['ROTA_ID'] = df['ROTA_ID'].fillna(-1)

# print(df)


## Exporting our dataset

Simply we're gonna export into a `.csv`.

In [ ]:
df.to_csv('../data/dataset.csv', sep=',', index=True, index_label='PES_ID')